In [ ]:
import gc
import os
import torch
import json
from tqdm.auto import tqdm
from internal.dataparsers.colmap_dataparser import Colmap, ColmapDataParser
from internal.utils.gaussian_model_loader import GaussianModelLoader
from internal.utils.light_gaussian import get_count_and_score, calculate_v_imp_score, get_prune_mask

In [ ]:
torch.autograd.set_grad_enabled(False)

In [ ]:
partition_base_dir = os.path.expanduser(
    "~/data/image_set/JNUCar_undistorted/colmap/drone/dense_max_2048/0/partitions-threshold_0.2/")

In [ ]:
dataparser_outputs = ColmapDataParser(
    os.path.join(partition_base_dir, ".."),
    output_path=os.getcwd(),
    global_rank=0,
    params=Colmap(
        appearance_groups="appearance_image_dedicated",
        eval_step=32
    ),
).get_outputs()

In [ ]:
image_name_to_camera_idx = {}
for idx, name in enumerate(dataparser_outputs.train_set.image_names):
    image_name_to_camera_idx[name] = idx
len(image_name_to_camera_idx)

In [ ]:
def get_cameras_by_image_list(image_list: list):
    cameras = []
    for i in image_list:
        cameras.append(dataparser_outputs.train_set.cameras[image_name_to_camera_idx[i]])
    return cameras

In [ ]:
partitions = torch.load(os.path.join(partition_base_dir, "partitions.pt"),
                        map_location="cpu")

orientation_transformation = partitions["orientation_transformation"]

model_paths = []
for idx, i in enumerate(partitions["ids"]):
    if len(partitions["image_indices"][idx]) < 32:
        continue
    model_paths.append((i, os.path.join(
        "../outputs/JNUAerial-0526/",
        f"P_{i[0]:03d}_{i[1]:03d}.txt")))
partitions.keys(), model_paths

In [ ]:
partition_id_to_index = {i: idx for idx, i in enumerate(partitions["ids"])}
partition_id_to_index

In [ ]:
dtype = torch.float
device = torch.device("cpu")

prune_percent = 0.6

n_gaussians_before_pruning = 0
n_gaussians_after_pruning = 0

with tqdm(model_paths) as t:
    for i in t:
        # if i[0] != (0, 0):
        #     continue

        partition_id = i[0]
        model_output_path = i[1]

        partition_xy = partitions["xys"][partition_id_to_index[i[0]]]
        load_file = GaussianModelLoader.search_load_file(i[1])
        t.set_description(f"{partition_xy}: {load_file}")

        # load ckpt file
        ckpt = torch.load(load_file, map_location=device)
        # instantiate GaussianModel
        gaussian_model = GaussianModelLoader.initialize_model_from_checkpoint(
            ckpt,
            device,
        )

        # get the position of all the Gaussians
        xyz = gaussian_model.get_means()

        # find those inside bounding box, pruning only apply to them
        reoriented_xyz = xyz @ orientation_transformation[:3, :3].T
        # include min bound, exclude max bound
        is_in_partition = torch.logical_and(torch.ge(reoriented_xyz[:, :2], partition_xy),
                                            torch.lt(reoriented_xyz[:, :2], partition_xy + 2 * partitions["radius"]))
        is_in_partition = torch.logical_and(is_in_partition[:, 0], is_in_partition[:, 1])

        # split into the inside and outside set
        outside_gaussians = {}
        inside_gaussians = {}
        for k, v in gaussian_model.properties.items():
            outside_gaussians[k] = v[~is_in_partition]
            inside_gaussians[k] = v[is_in_partition]

        # update gaussian model, containing only inside set
        gaussian_model.properties = inside_gaussians
        # move gaussian model to cuda
        gaussian_model.cuda()

        n_gaussians_before_pruning += gaussian_model.n_gaussians

        # get partition image list
        with open(os.path.join(model_output_path, "cameras.json"), "r") as f:
            cameras_json = json.load(f)
        image_list = [i["img_name"] for i in cameras_json]

        cameras = get_cameras_by_image_list(image_list)

        # calculate scores
        hit_camera_count, opacity_score, alpha_score, visibility_score = get_count_and_score(
            gaussian_model,
            cameras,
            anti_aliased=True,
        )

        # prune by visibility
        # # get prune indices
        # visibility_score_close_to_zero = torch.isclose(visibility_score, torch.tensor(0.).to(visibility_score))
        # visibility_score_close_to_zero_count = visibility_score_close_to_zero.sum()
        # prune_percent = 0.9
        # # ignore the Gaussians visibility score close zero
        # keep_count = ((visibility_score.shape[0] - visibility_score_close_to_zero_count) * (1 - prune_percent)).to(torch.int)
        # # get the indices (in partition) to be preserved
        # visibility_score_sorted = torch.sort(visibility_score, descending=True)
        # local_indices_to_preserved = visibility_score_sorted.indices[:keep_count].cpu()

        # prune by opacity
        v_imp_score = calculate_v_imp_score(gaussian_model.get_scaling, opacity_score, 0.1)
        prune_mask = get_prune_mask(prune_percent, v_imp_score)
        # local_indices_to_preserved = (~prune_mask).nonzero().squeeze(-1).cpu()

        # prune gaussian model with the LightGaussian mask
        gaussian_model.properties = {k: v[~prune_mask] for k, v in gaussian_model.properties.items()}
        n_gaussians_after_pruning += gaussian_model.n_gaussians
        # # the indices (in partition) to be pruned
        # local_indices_to_prune = visibility_score_sorted.indices[keep_count:]
        # is_in_partition_indices = is_in_partition.nonzero().squeeze(-1)
        # gaussian_indices_to_prune = is_in_partition_indices[local_indices_to_prune.to(device=is_in_partition_indices.device)]
        # 
        # # convert prune indices to preserve mask
        # preserve_mask = torch.ones_like(is_in_partition, dtype=torch.bool)
        # preserve_mask[gaussian_indices_to_prune] = False

        # update state_dict of checkpoint
        """
        [NOTE]
        The codes related to the `static part` below have not been released yet.
        So, rather than move some Gaussian to `static part`, you should prune the `state_dict`, `optimizer_states` and `gaussian_model_extra_state_dict` of `ckpt` according to the mask `local_indices_to_preserved`.
        """
        # for i in state_dict:
        #     # move those Gaussian outside the partition to static part, which will not be optimized during finetune
        #     static_gaussian_property_key = f"static_{i}"
        #     static_gaussian_property = ckpt["state_dict"][i][~is_in_partition]
        #     if static_gaussian_property_key in ckpt["state_dict"]:
        #         original_static_gaussian_num = ckpt["state_dict"][static_gaussian_property_key].shape[0]
        #         new_static_gaussian_num = static_gaussian_property.shape[0]
        #         static_gaussian_property = torch.concat([
        #             ckpt["state_dict"][static_gaussian_property_key],
        #             static_gaussian_property,
        #         ], dim=0)
        #         print(f"#{partition_id}: {original_static_gaussian_num} static Gaussians exists, merge with {new_static_gaussian_num} new static Gaussians, total {static_gaussian_property.shape[0]} after merging")
        #     ckpt["state_dict"][static_gaussian_property_key] = static_gaussian_property
        #     # make optimizable Gaussians only contains those locating in partition
        #     ckpt["state_dict"][i] = state_dict[i]

        # merge inside and outside
        gaussian_model.to(device=device)
        gaussian_model.properties = {k: torch.concat(
            [outside_gaussians[k], v],
            dim=0,
        ) for k, v in gaussian_model.properties.items()}
        
        # update gaussian states of ckpt
        for k, v in gaussian_model.state_dict().items():
            state_dict_full_key = "gaussian_model.{}".format(k)
            assert state_dict_full_key in ckpt["state_dict"]
            ckpt["state_dict"][state_dict_full_key] = v

        # prune optimizer state
        optimizer_prunable_states = ["exp_avg", "exp_avg_sq"]
        property_names = list(gaussian_model.property_names)
        new_optimizer_state_list = []
        for optimizer_state in ckpt["optimizer_states"]:
            if len(property_names) == 0:
                break

            for param_group_idx, param_group in enumerate(optimizer_state["param_groups"]):
                # whether a gaussian param_group
                param_group_name = param_group["name"]
                if param_group_name not in property_names:
                    continue

                outside_gaussian_states = {
                    k: optimizer_state["state"][param_group_idx][k][~is_in_partition]
                    for k in optimizer_prunable_states
                }
                inside_gaussian_states = {
                    k: optimizer_state["state"][param_group_idx][k][is_in_partition][~prune_mask.to(device=device)]
                    for k in optimizer_prunable_states
                }

                # replace optimizer states
                for k in optimizer_prunable_states:
                    optimizer_state["state"][param_group_idx][k] = torch.concat(
                        [outside_gaussian_states[k], inside_gaussian_states[k]],
                        dim=0,
                    )

                property_names.remove(param_group_name)

        # prune density controller state_dict by simply replacing with zeros
        for i in ckpt["state_dict"]:
            if i.startswith("density_controller."):
                ckpt["state_dict"][i] = torch.zeros((gaussian_model.n_gaussians, *ckpt["state_dict"][i].shape[1:]))

        # save checkpoint
        checkpoint_save_dir = os.path.join(model_output_path, "pruned_checkpoints")
        os.makedirs(checkpoint_save_dir, exist_ok=True)
        torch.save(ckpt, os.path.join(checkpoint_save_dir, f"latest-opacity_pruned-{prune_percent}.ckpt"))
        
        del ckpt
        del gaussian_model
        del outside_gaussians
        del inside_gaussians
        gc.collect()
        torch.cuda.empty_cache()

f"{n_gaussians_after_pruning} / {n_gaussians_before_pruning}"